<a href="https://colab.research.google.com/github/jmgold/DEI-Collections/blob/main/Booklist_Holdings_Comparison.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Jeremy Goldstein
Minuteman Library Network

This Python script will compare an excel or csv file of local holdings to a curated booklist to find how much of that list you own and which titles are missing based on fuzzy matching titles and authors.

The script may appear daunting at first glance but it will do all the work for you.  Simply hit play, upload a couple spreadsheets where needed and enter a match value in step 4 when prompted.

Minuteman staff may view [a video demoing the script here.](https://drive.google.com/file/d/1KP-7dD0OAgjdFaK14myptWCRrhmwuXPS/view?usp=sharing)

# Instructions

**Prerequisite**

You will need two files containing at least title and author information.  The file can be a csv or excel (either .xls or .xlsx is fine).  Titles and authors need to each be in their own column with a header of 'title' and 'author' (capitalization does not matter).  The files can contain as many additional columns as you wish, so long as each one does include its own column header.
The first file will be a list of titles you wish to check against your current holdings.  The second should be a list of your current holdings.

**Running the script**

Each code block can be run one at a time by clicking the play icon that appears when you hover over the [] marker.  Once that block finishes running a green check will appear to the left of the block and any output from that portion of the script will appear beneath it, along with any errors that may be encountered.

You may also use the "Run All" function, found within the Runtime drop down menu to Run the entire script, though there are points where user input is required before the script will continue past a particular code block to be on the lookout for.  These are indicated by the text **Action Required**.  There are also a few **Optional** code blocks where you can download the output of the script to excel.

You may reset the output by going to the Edit menu and selecting clear all outputs.  You can also review files that have been uploaded or created as part of this script using the folder icon i the left hand navigation menu. 

# Step 1: Configure Python/Colab Environment


In [ ]:
#@title
#nmslib is not included by default with Colab and must be installed.
!pip install nmslib
!pip install xlrd==1.2.0

#import Python libraries that will be used within the script
import pandas as pd
import io
import numpy as np
import os
import re
import nmslib
import altair as alt
from sklearn.feature_extraction.text import TfidfVectorizer
%load_ext google.colab.data_table
from google.colab import files

# Step 2: Import data
Script will load tabular data into two dataframes using the Pandas library for Python.  

**Action Required** Upload either a csv or excel file with the list of suggested titles you wish to match your holdings against.  

Must contain at least 2 columns with the headers author and title (capitalization does not matter).

In [ ]:
#@title
uploaded_list = files.upload()

for fn in uploaded_list.keys():
  file_name=fn
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded_list[fn])))

#Loads the uploaded file into a datafrome or returns an error if the file is an incorrect format  
if file_name.endswith('.csv'):
  booklist_df = pd.read_csv(io.BytesIO(uploaded_list[file_name]))
elif file_name.endswith('.xls'):
  booklist_df = pd.read_excel(io.BytesIO(uploaded_list[file_name]))
elif file_name.endswith('.xlsx'):
  booklist_df = pd.read_excel(io.BytesIO(uploaded_list[file_name]))
else:
  print("error file is not .csv or excel")

#clean up data in the table and add BooklistMatch Column for use later
#change null to blank
booklist_df=booklist_df.fillna('')
#change headers to lower case
booklist_df.columns = [x.lower() for x in booklist_df.columns]
#save list of column headers for use at the end
booklist_headers = booklist_df.columns.tolist()
#create match point column
booklist_df['BooklistMatch']=booklist_df['author']+booklist_df['title']

**Optional** Preview the contents of this dataframe.  
You can use the Filter button to further explore the data.

In [ ]:
#@title
#Preview booklist dataframe
booklist_df

**Action Required** Upload an excel or csv file containing the titles within your holdings

Must contain at least 2 columns with the headers author and title (capitalization does not matter)

In [ ]:
#@title
uploaded_holdings = files.upload()

for fn in uploaded_holdings.keys():
  file_name=fn
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded_holdings[fn])))

#Loads the uploaded file into a datafrome or returns an error if the file is an incorrect format
if file_name.endswith('.csv'):
  holdings_df = pd.read_csv(io.BytesIO(uploaded_holdings[file_name]))
elif file_name.endswith('.xls'):
  holdings_df = pd.read_excel(io.BytesIO(uploaded_holdings[file_name]))
elif file_name.endswith('.xlsx'):
  holdings_df = pd.read_excel(io.BytesIO(uploaded_holdings[file_name]))
else:
  print("error file is not .csv or excel")

#clean up data in the table and add BooklistMatch Column for use later
#remove null values
holdings_df=holdings_df.fillna('')
#change headers to lower case
holdings_df.columns = [x.lower() for x in holdings_df.columns]
#save list of column headers for use at the end
holdings_headers = holdings_df.columns.tolist()
#Create MatchPoint field
holdings_df['HoldingsMatch']=holdings_df['author']+holdings_df['title']


**Optional** Preview holdings data.

**Note:** If the file is contains more than 20,000 rows the preview will be displayed different than the prior preview and will not include the browse features.

In [ ]:
#@title
#Preview data
holdings_df


# Step 3: Calculate Matches Dataframe
Compare two data frames and calculate match confidence value for each pair of rows.

Matching algorithm is taken from [Fuzzy Matching at Scale by Josh Taylor](https://towardsdatascience.com/fuzzy-matching-at-scale-84f2bfd0c536) (viewed 11/24/2021)

In [ ]:
#@title
def ngrams(string, n=3):
    """Takes an input string, cleans it and converts to ngrams. """
    string = str(string)
    string = string.lower() # lower case
    string = string.encode("ascii", errors="ignore").decode() #remove non ascii chars
    chars_to_remove = [")","(",".","|","[","]","{","}","'","-"]
    rx = '[' + re.escape(''.join(chars_to_remove)) + ']' #remove punc, brackets etc...
    string = re.sub(rx, '', string)
    string = re.sub(' +',' ',string).strip() # get rid of multiple spaces and replace with a single
    string = ' '+ string +' ' # pad names for ngrams...
    ngrams = zip(*[string[i:] for i in range(n)])
    return [''.join(ngram) for ngram in ngrams]


###used to build the matching table
##### Create a list of items to match here:
booklist_match = list(booklist_df["BooklistMatch"].unique())
#Building the TFIDF off the clean dataset
vectorizer = TfidfVectorizer(min_df=1, analyzer=ngrams)
tf_idf_matrix = vectorizer.fit_transform(booklist_match)

##### Create a list of messy items to match here:
holdings_match = list(holdings_df["HoldingsMatch"].unique()) #unique list of names


messy_tf_idf_matrix = vectorizer.transform(holdings_match)

# create a random matrix to index
data_matrix = tf_idf_matrix#[0:1000000]

# Set index parameters
# These are the most important ones
M = 80
efC = 1000

num_threads = 4 # adjust for the number of threads
# Intitialize the library, specify the space, the type of the vector and add data points 
index = nmslib.init(method='simple_invindx', space='negdotprod_sparse_fast', data_type=nmslib.DataType.SPARSE_VECTOR) 

index.addDataPointBatch(data_matrix)
# Create an index
index.createIndex() 


# Number of neighbors 
num_threads = 4
K=1
query_matrix = messy_tf_idf_matrix
query_qty = query_matrix.shape[0]
nbrs = index.knnQueryBatch(query_matrix, k = K, num_threads = num_threads)

mts =[]
for i in range(len(nbrs)):
  original_nm = holdings_match[i]
  try:
    matched_nm   = booklist_match[nbrs[i][0][0]]
    conf         = nbrs[i][1][0]
  except:
    matched_nm   = "no match found"
    conf         = None
  mts.append([original_nm,matched_nm,conf])

mts = pd.DataFrame(mts,columns=['holdings_match','booklist_match','conf'])
#change negative values to positive for ease of reading
mts['conf'] = mts['conf'].abs()

#Step 4: Determine matches

**Action Required** In the table below you can browse the results to see the match confidence scores assigned to each pair.

Look at values of the 'conf' column to find the point at which you feel the entries are matched correctly.  Generally we recommend looking at the values between .5 and .7 as a starting point.  To help here you can see a preview of the matched data that has been limited to the relevant range of values.

In [ ]:
#@title
#.66 used as default match confidence value based on initial testing, will be overwritten in step below

mts = mts.sort_values(by=['conf'])
mts[mts['conf'].between(.5,.7)]

**Optional** Download matching table to an Excel file

In [ ]:
#@title
mts.to_excel("match_confidence.xlsx")
files.download('/content/match_confidence.xlsx')

**Action Required** Enter the confidence value you wish to use for determining correct matches, based on the output from the prior code block.  

Any value greater than or equal the number you enter will be considered a match.

Once this code block has been run once, you may adjust the slider without having to run it again.

In [ ]:
#@title Set Confidence Value { run: "auto", vertical-output: true, display-mode: "form" }
match_confidence = 0.66 #@param {type:"slider", min:0, max:1, step:0.001}

**Percentage of Titles from list that are in your collection**

In [ ]:
#@title
pct_held = (len(mts.loc[mts['conf'] >= match_confidence]) / len(booklist_df)) * 100
source = pd.DataFrame({"titles": ["all","all"],"category": ["held","not held"], "value":[pct_held,100-pct_held]})

print('You own '+str(round(pct_held, 2))+'% of the titles in the booklist')
alt.Chart(source).mark_bar().encode(
    x='sum(value)',
    y='titles',
    color='category',
    order = alt.Order('category',sort='ascending')
)

**Titles Found in Your Collection**


In [ ]:
#@title
#Merge the holdings dataframe with the instances where a match has been found
found_results = holdings_df.reset_index().merge(mts.loc[mts['conf'] >= match_confidence], left_on='HoldingsMatch', right_on='holdings_match').set_index('index')
#found_results = found_results.rename(columns=str.capitalize)
found_results[holdings_headers]

**Optional** Download results to an Excel file

In [ ]:
#@title
found_results[holdings_headers].to_excel("found_results.xlsx")
files.download('/content/found_results.xlsx')

**Titles Not in your collection**

In [ ]:
#@title
#Create missing dataframe containing the titles from the booklist that were not matched to your holdings
booklist_found = booklist_df.reset_index().merge(mts.loc[mts['conf'] >= match_confidence], left_on='BooklistMatch', right_on='booklist_match').set_index('index')
common = booklist_df.merge(booklist_found, how='outer', left_index=True, right_index=True)
common = common[common[['conf']].notna().all(axis=1)]
missing = booklist_df.merge(common, how='outer', left_index=True, right_index=True)

missing = missing[missing['conf'].isnull()][booklist_headers]
missing

**Optional** Download missing titles to an Excel file

In [ ]:
#@title
missing.to_excel("missing_results.xlsx")
files.download('/content/missing_results.xlsx')